In [ ]:
# use numba to speed up execution of the function
import numpy as np
from numpy import log, sqrt, exp
from scipy.stats import norm
from numba import jit

@jit
def blackscholes(S, K, T, r, σ):
  d1 = (log(S/K) + (r + σ**2 / 2) * T) / (σ*sqrt(T))
  d2 = d1 - σ * sqrt(T)
  call = S * norm.cdf(d1) - K * exp(-r*T) * norm.cdf(d2)
  return call

<ipython-input-32-46d783c5a084>:8: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def blackscholes(S, K, T, r, σ):


In [ ]:
# generate synthetic data in the vicinity of S=100, K=100, T=1, etc. and save it to disk
SK = np.arange(99, 101, 0.05)
T = np.arange(0.9, 1.1, 0.05)
r = np.arange(0.02, 0.05, 0.001)
σ = np.arange(0.1, 0.2, 0.01)
SS, KK, TT, rr, σσ = np.meshgrid(SK, SK, T, r, σ)
YY = blackscholes(SS, KK, TT, rr, σσ)

rows = np.stack([z.ravel() for z in (YY, SS, KK, TT, rr, σσ)], axis=1)
np.savetxt('data.csv', rows, fmt='%.5f', delimiter=',', newline='\n', comments='', header='Call,S,K,T,r,sigma')

<ipython-input-32-46d783c5a084>:7: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "blackscholes" failed type inference due to: Untyped global name 'norm': Cannot determine Numba type of <class 'scipy.stats._continuous_distns.norm_gen'>

File "<ipython-input-32-46d783c5a084>", line 11:
def blackscholes(S, K, T, r, σ):
    <source elided>
  d2 = d1 - σ * sqrt(T)
  call = S * norm.cdf(d1) - K * exp(-r*T) * norm.cdf(d2)
  ^

  @jit
/usr/local/lib/python3.10/dist-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "blackscholes" was compiled in object mode without forceobj=True.

File "<ipython-input-32-46d783c5a084>", line 8:
@jit
def blackscholes(S, K, T, r, σ):
^

  warnings.warn(errors.NumbaWarning(warn_msg,
/usr/local/lib/python3.10/dist-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected. This is 

In [ ]:
# we generated 120MB of data !
!ls -l data.csv
!head data.csv

-rw-r--r-- 1 root root 122466281 Feb  9 13:29 data.csv
Call,S,K,T,r,sigma
4.66163,99.00000,99.00000,0.90000,0.02000,0.10000
5.02648,99.00000,99.00000,0.90000,0.02000,0.11000
5.39224,99.00000,99.00000,0.90000,0.02000,0.12000
5.75866,99.00000,99.00000,0.90000,0.02000,0.13000
6.12558,99.00000,99.00000,0.90000,0.02000,0.14000
6.49287,99.00000,99.00000,0.90000,0.02000,0.15000
6.86043,99.00000,99.00000,0.90000,0.02000,0.16000
7.22818,99.00000,99.00000,0.90000,0.02000,0.17000
7.59606,99.00000,99.00000,0.90000,0.02000,0.18000


In [ ]:
# train a MLP on the data and save trained model to disk
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from joblib import dump, load

df = pd.read_csv("data.csv", dtype=np.float32)
y = df.pop('Call').values
df.S = df.S/100
df.K = df.K/100
X = df.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
regr = MLPRegressor(random_state=42, max_iter=500).fit(X_train, y_train)
regr.predict(X_test[:5])
regr.score(X_test, y_test)
dump(regr, 'mlpregressor.joblib')

['mlpregressor.joblib']

In [ ]:
# create input data for inference, in the range and way out of range, model should behave badly there
!echo "S,K,T,r,sigma" > input.csv
!echo "100,100,1,0.05,0.1" >> input.csv
!echo "100,100,1,0.05,0.11" >> input.csv
!echo "100,100,1,0.05,0.12" >> input.csv
!echo "99.8,100,1,0.05,0.11" >> input.csv
!echo "99.8,105,1,0.02,0.2" >> input.csv
!echo "95,125,0.5,0.02,0.2" >> input.csv

In [ ]:
# reload the model and run inference on new data
# ground truth is 6.8020, 7.1543, 7.5100, 7.0121, 6.2715, 0.1829
regr = load('mlpregressor.joblib')
df = pd.read_csv('input.csv')
print(df)
df.S = df.S/100
df.K = df.K/100
regr.predict(df.values)

       S    K    T     r  sigma
0  100.0  100  1.0  0.05   0.10
1  100.0  100  1.0  0.05   0.11
2  100.0  100  1.0  0.05   0.12
3   99.8  100  1.0  0.05   0.11
4   99.8  105  1.0  0.02   0.20
5   95.0  125  0.5  0.02   0.20


array([ 6.73918979,  7.09937476,  7.45955973,  6.96970988,  6.11336577,
       -9.57364573])